In [1]:
import tarfile
import os

import numpy as np
import boto3
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [2]:
spencer = boto3.session.Session(profile_name='spencer')

In [3]:
spencer.client('sts').get_caller_identity().get('Account')

'652081700929'

In [4]:
boto3.client('sts').get_caller_identity().get('Account')

'756586798028'

In [5]:
spencer_s3 = spencer.client("s3")

In [ ]:
spencer_s3.download_file('sagemaker-us-east-1-652081700929', "final-distilbert-regression-800k-2020-06-15-07-04-51-437/output/model.tar.gz", "model.tar.gz")

In [ ]:
with tarfile.open("model.tar.gz", "r:gz") as tar:
    tar.extractall(path="model")

In [ ]:
# os.remove("model.tar.gz")

In [6]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased', use_fast=True)
model = TFAutoModelForSequenceClassification.from_pretrained("model")

In [7]:
text = "this place is amazing!"

In [27]:
def predict(text):
    return model(tokenizer.encode(text, return_tensors='tf', max_length=512))[0].numpy()[0][0]

def scale(pred):
    return (pred - 1) * .25

def squish(scaled):
    return np.clip(scaled, 0, 1)

def pred_scale_and_squish(text):
    pred = predict(text)
    scaled = scale(pred)
    squished = squish(scaled)
    
    return pred, scaled, squished

In [28]:
predict(text)

5.074088

In [40]:
bad_text = "this place is the worst!!!"

In [41]:
predict(bad_text)

1.004437

In [31]:
%%timeit
predict(text)

101 ms ± 9.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
s3 = boto3.client("s3")

In [ ]:
s3.upload_file(Filename='model.tar.gz', Bucket='yelpsense', Key='models/sentiment/distilbert/regression/model.tar.gz')

In [32]:
pred_scale_and_squish(text)

(5.074088, 1.018522024154663, 1.0)

In [42]:
pred_scale_and_squish(bad_text)

(1.004437, 0.0011092424392700195, 0.0011092424392700195)

In [11]:
text = "this place is kinda good"

In [12]:
pred_scale_and_squish(text)

(3.8772535, 0.7754507064819336, 0.6938626766204834)

In [13]:
squish(3.8772535)

1.0

In [16]:
tokenizer2 = AutoTokenizer.from_pretrained(
    'distilbert-base-uncased', use_fast=True)
model2 = TFAutoModelForSequenceClassification.from_pretrained(
    "spentaur/yelp")


In [17]:
def predict2(text):
    return model2(tokenizer2.encode(text, return_tensors='tf', max_length=512))[0].numpy()[0][0]

In [43]:
%%timeit
predict2(text)

100 ms ± 6.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%%timeit
predict2(text)

102 ms ± 9.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [45]:
%%timeit
predict(text)

95.8 ms ± 3.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
squish(predict2(text))

1.0

In [46]:
pred_scale_and_squish("alright")

(2.997344, 0.49933600425720215, 0.49933600425720215)

In [7]:
review = """This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone.
This restaurant is one of the bests in Champaign. Coming from the NYC/Philly/Princeton area, Ive found few places that really mimic the food quality and scene. There chicken sandwiches are great, especially the Nashville hot. The sauces are great too, I really like the honey chili sauce. On Tuesdays they have Tacos as specials and there chicken taco is amazing! There appetizers are also tasty, my favorite is the Brussels sprouts. I wish they would add it to there regular menu. Its so fresh and very flavorful. As a plus, they also have non-alcoholic mock tails for everyone."""

In [2]:
from transformers import pipeline

In [3]:
ts = pipeline("summarization", model="t5-large", tokenizer="t5-large", framework="tf")

In [ ]:
ts(review, min_length=42, clean_up_tokenization_spaces=True, max_length=1024)[0]['summary_text']

Token indices sequence length is longer than the specified maximum sequence length for this model (3314 > 512). Running this sequence through the model will result in indexing errors
